<a href="https://colab.research.google.com/github/sominshim/DL_beginner/blob/main/%EC%8B%A4%EC%8A%B53.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DL_4_3_multi_layers.py

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
type(mnist)

tuple

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist

In [ ]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(60000, 28, 28) (10000, 28, 28)
(60000,) (10000,)


### 3차원에서 2차원으로 변경하기

In [ ]:
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

In [ ]:
print(x_train.shape, x_test.shape)

(60000, 784) (10000, 784)


In [ ]:
28*28

784

In [ ]:
# minmax scale
x_train = x_train/255
x_test = x_test/255

## Quiz
mnist 데이터에 대해 동작하는 모델을 구축하세요.

In [ ]:
def softmax_regression_onehot(x, y):
    model = tf.keras.Sequential()

    # 학습할 때 각 레이어는 독립적이야 한다. 
    # 이를 위해서 non-linear 함수(relu)를 넣는다. > 앞에서 나오는 값에 변형을 준다.
    # softmax, sigmoid  같은 활성화 함수는 weight 값을 0에 가깝게 수렴한다. 
    # 때문에 학습이 안된다. > banishing
    model.add(tf.keras.layers.Input(shape=[784]))
    # (100, 256) = (100, 784) @ (784, 256) 
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    # (100, 256) = (100, 256) @ (256, 256) 
    model.add(tf.keras.layers.Dense(256, activation='softmax')) 
    # (100, 10) = (100, 256) @ (256, 10) 
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.summary()    

    # optimizer = SGD, Adam, RMSProp
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics='acc')

    model.fit(x, y, epochs=10, batch_size=100, verbose=1, 
              validation_split=0.2) # default batch=32 

    return model

In [ ]:
model = softmax_regression_onehot(x_train, y_train)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 256)               200960    
                                                                 
 dense_9 (Dense)             (None, 256)               65792     
                                                                 
 dense_10 (Dense)            (None, 256)               65792     
                                                                 
 dense_11 (Dense)            (None, 10)                2570      
                                                                 
Total params: 335,114
Trainable params: 335,114
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
480/480 [==============================] - 2s 4ms/step - loss: 0.6733 - acc: 0.8142 - val_loss: 0.2351 - val_acc: 0.9389
Epoch 2/10
480/480 [============================

In [ ]:
# Adam
model.evaluate(x_test, y_test, verbose=0)

[0.26526620984077454, 0.9257000088691711]

In [ ]:
# RMSProp
model.evaluate(x_test, y_test, verbose=0)

[0.09660395234823227, 0.9769999980926514]

In [ ]:
# validation used / val_loss 가 점차 줄어들고 있는가? 확인. 
# 이 val_loss 를 보고 튜닝
# (100, 10) = (100, 784) @ (784, 10) 
model = softmax_regression_onehot(x_train, y_train)
model.evaluate(x_test, y_test, verbose=0)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                7850      
                                                                 
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
480/480 [==============================] - 2s 3ms/step - loss: 0.6248 - acc: 0.8449 - val_loss: 0.3461 - val_acc: 0.9055
Epoch 2/10
480/480 [==============================] - 1s 3ms/step - loss: 0.3414 - acc: 0.9054 - val_loss: 0.3002 - val_acc: 0.9176
Epoch 3/10
480/480 [==============================] - 1s 3ms/step - loss: 0.3097 - acc: 0.9138 - val_loss: 0.2857 - val_acc: 0.9202
Epoch 4/10
480/480 [==============================] - 1s 3ms/step - loss: 0.2950 - acc: 0.9178 - val_loss: 0.2779 - val_acc: 0.9231
Epoch 5/10
480/480 [==============================] 

[0.27344775199890137, 0.9259999990463257]

# DL_4_4_cnn_basic.py

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
def softmax_regression_onehot(x, y):
    # Sequential 모델링 문법
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Input(shape=[28, 28, 1]))
    model.add(tf.keras.layers.Conv2D(filters=6, kernel_size=[5, 5], strides=1, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2,padding='same'))
    model.add(tf.keras.layers.Conv2D(filters=10, kernel_size=[5, 5], strides=1, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2,padding='same'))
    model.add(tf.keras.layers.Flatten())
    # model.add(tf.keras.layers.Dense(128, activation='relu')) 
    # Dense layer를 추가한 이유: output 10개를 추출하기 전에 하나의 단계를 더 넣음.
    # -> 하지만 모델이 무거워질 수록/ 깊어질 수록 Dense layer(는 parameter 값이 너무 많아짐)를 굳이 쓰지 않아도 성능이 잘나온다.

    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.summary()    

    # optimizer = SGD, Adam, RMSProp
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics='acc')

    model.fit(x, y, epochs=10, batch_size=100, verbose=1, 
              validation_split=0.2) # default batch=32 

    return model

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist

In [ ]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(60000, 28, 28) (10000, 28, 28)
(60000,) (10000,)


In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [ ]:
# minmax scale
x_train = x_train/255
x_test = x_test/255

In [ ]:
model = softmax_regression_onehot(x_train, y_train)
model.evaluate(x_test, y_test, verbose=0)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 6)        0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 10)        1510      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 7, 10)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 490)               0         
                                                                 
 dense_2 (Dense)             (None, 128)              

[0.04618501290678978, 0.9847000241279602]

# DL_4_5_vgg16.py

https://github.com/tensorflow/models/blob/master/research/slim/nets/vgg.py 

In [ ]:
import tensorflow as tf

# functional 모델링 문법
inputs = tf.keras.layers.Input(shape=[224, 224, 3])
x = tf.keras.layers.Conv2D(64, [3, 3], 1, 'same', activation='relu')(inputs)
x = tf.keras.layers.Conv2D(64, [3, 3], 1, 'same', activation='relu')(x)
x = tf.keras.layers.MaxPool2D([2, 2], 2)(x)

x = tf.keras.layers.Conv2D(128, [3, 3], 1, 'same', activation='relu')(x)
x = tf.keras.layers.Conv2D(128, [3, 3], 1, 'same', activation='relu')(x)
x = tf.keras.layers.MaxPool2D([2, 2], 2)(x)

x = tf.keras.layers.Conv2D(256, [3, 3], 1, 'same', activation='relu')(x)
x = tf.keras.layers.Conv2D(256, [3, 3], 1, 'same', activation='relu')(x)
x = tf.keras.layers.Conv2D(256, [3, 3], 1, 'same', activation='relu')(x)
x = tf.keras.layers.MaxPool2D([2, 2], 2)(x)

x = tf.keras.layers.Conv2D(512, [3, 3], 1, 'same', activation='relu')(x)
x = tf.keras.layers.Conv2D(512, [3, 3], 1, 'same', activation='relu')(x)
x = tf.keras.layers.Conv2D(512, [3, 3], 1, 'same', activation='relu')(x)
x = tf.keras.layers.MaxPool2D([2, 2], 2)(x)

x = tf.keras.layers.Conv2D(512, [3, 3], 1, 'same', activation='relu')(x)
x = tf.keras.layers.Conv2D(512, [3, 3], 1, 'same', activation='relu')(x)
x = tf.keras.layers.Conv2D(512, [3, 3], 1, 'same', activation='relu')(x)
x = tf.keras.layers.MaxPool2D([2, 2], 2)(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(4096, activation='relu')(x)
x = tf.keras.layers.Dense(4096, activation='relu')(x)
x = tf.keras.layers.Dense(1000, activation='relu')(x)

model = tf.keras.Model(inputs, x)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_22 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_23 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_24 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_25 (Conv2D)          (None, 112, 112, 128)     147584    
                                                           

# Google Inception

inception_v4.py

https://github.com/tensorflow/models/blob/master/research/slim/nets/inception_v4.py

### Example) 5X5 이미지가 있을 때


0 0 0 0 0

0 0 0 0 0

0 0 0 0 0

0 0 0 0 0

0 0 0 0 0



        5X5 = 1  
        --> 25

        3X3 = 3X3
                  3X3 = 1
        --> 9+9 = 18

        1X5 = 5X1
                 5X1 = 1
        --> 5+5 = 10

# DL_4_6_dogs_vs_cats.py

In [1]:
import tensorflow as tf

In [2]:
gen_train = tf.keras.preprocessing.image.ImageDataGenerator() # 학습용
gen_valid = tf.keras.preprocessing.image.ImageDataGenerator() # 튜닝용

In [3]:
gen_train = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255, 
                                                                                                horizontal_flip=True,
                                                                                                zoom_range=0.2,
                                                                                                width_shift_range=0.2)
gen_valid = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255) # arguemetation 하지 않는다.

## 이미지 읽어오기

In [5]:
import pandas as pd

flow_train = gen_train.flow_from_directory('/content/drive/MyDrive/디지털오픈랩 딥러닝 교육/data/dogs_vs_cats_small/train',
                                           target_size=[150, 150],
                                           batch_size=32,
                                           class_mode='binary') 
flow_valid = gen_valid.flow_from_directory('/content/drive/MyDrive/디지털오픈랩 딥러닝 교육/data/dogs_vs_cats_small/valid',
                                           target_size=[150, 150],
                                           batch_size=32,
                                           class_mode='binary') 
flow_test = gen_valid.flow_from_directory('/content/drive/MyDrive/디지털오픈랩 딥러닝 교육/data/dogs_vs_cats_small/test',
                                           target_size=[150, 150],
                                           batch_size=32,
                                           class_mode='binary') 

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [6]:
def softmax_regression_onehot(xy, valid):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Input(shape=[150, 150, 1]))
    model.add(tf.keras.layers.Conv2D(filters=6, kernel_size=[3, 3], strides=1, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2,padding='same'))
    model.add(tf.keras.layers.Conv2D(filters=10, kernel_size=[3, 3], strides=1, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2,padding='same'))
    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.summary()    

    # optimizer = SGD, Adam, RMSProp
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                loss=tf.keras.losses.binary_crossentropy,
                metrics='acc')

    model.fit(xy, epochs=10, verbose=1, validation_data=valid) 

    return model

In [ ]:
model = softmax_regression_onehot(flow_train, flow_valid)
model.evaluate(x_test, y_test, verbose=0)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 6)       60        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 6)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 10)        550       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 38, 10)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 14440)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [ ]:
# 모델 저장
model.save('model/dog.h5')
model = tf.keras.models.load_model('model/dogs.h5')

## Keras model load

In [ ]:
conv_base = tf.keras.applications.VGG16(include_top=False, # 마지막 output layer 가져오지 않는다.
                                        input_shape=[150, 150, 3])

In [ ]:
conv_base.trainable = False